In [8]:
import requests
import json
api_key = "6e629535df364ba79b406daf6a34adac"

# Search for crime-related news articles in Karnataka
url = f"https://newsapi.org/v2/everything?q=crime+in+Karnataka&apiKey={api_key}"
response = requests.get(url)
articles = response.json()["articles"]

# Extract the relevant information from the articles
# Extract the relevant information from the articles
crime_data = []
for article in articles:
    crime_city = None
    crime_place = None
    
    # Extract the city and place of the crime from the article's description
    description = article["description"]
    if " in " in description:
        parts = description.split(" in ")
        crime_place = parts[0].strip()
        crime_city = parts[1].strip()
    
    crime_data.append({
        "title": article["title"],
        "description": article["description"],
        "date": article["publishedAt"],
        "location": "Karnataka",
        "crime_city": crime_city,
        "crime_place": crime_place
    })

# Save the crime data to a file
with open("crime_data.json", "w") as f:
    f.write(json.dumps(crime_data))

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load the preprocessed crime data
with open("crime_data.json", "r") as f:
    crime_data = json.loads(f.read())

# Convert the crime data into a pandas DataFrame
df = pd.DataFrame(crime_data)
df.head()

,title,description,date,location,crime_city,crime_place
0,Solving Crime | From IIT grad to wanted crimin...,"In seven years, Nagendra Reddy turned from an ...",2024-01-20T06:21:49Z,Karnataka,2011 when he was shot dead,"In seven years, Nagendra Reddy turned from an ..."
1,Gauri Lankesh case: 3 key accused identified b...,Among the witnesses are a cab driver and a mob...,2024-01-18T04:18:20Z,Karnataka,the building,Among the witnesses are a cab driver and a mob...
2,Karnataka BJP MP Pratap Simha’s brother held f...,"In a press statement, the Forest department of...",2023-12-30T16:50:16Z,Karnataka,None,None
3,"January 10, 1984, Forty Years Ago: PM-Abdullah...",A 50-minute meeting between Prime Minister Ind...,2024-01-10T01:00:15Z,Karnataka,None,None
4,"As BJP seeks K’taka governor’s intervention, S...",Citing the Lokayukta arrest of BS Yediyurappa ...,2024-01-03T13:18:00Z,Karnataka,None,None


In [24]:
# Define the features and the target variable
# Define the features and the target variable
X = df[["title", "description", "date"]]
y = df["location"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the text data into numerical format using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train["title"] + X_train["description"])
X_test_vectorized = vectorizer.transform(X_test["title"] + X_test["description"])


In [25]:
# Train the Multinomial Naive Bayes model for crime prediction
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

MultinomialNB()

In [26]:

# Make predictions on the test set
y_pred = model.predict(X_test_vectorized)


In [27]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [31]:
# Make predictions for a specific date
date = "2023-03-20"
new_data = {"title": [], "description": [], "date": [date], "location": [], "crime_city": [], "crime_place": []}

# Search for crime-related news articles on the specific date
url = f"https://newsapi.org/v2/everything?q=crime+in+Karnataka+AND+{date}&apiKey={api_key}"
response = requests.get(url)
articles = response.json()["articles"]

# Extract the relevant information from the articles
for article in articles:
    crime_city = None
    crime_place = None
    
    # Extract the city and place of the crime from the article's description
    description = article["description"]
    if " in " in description:
        parts = description.split(" in ")
        crime_place = parts[0].strip()
        crime_city = parts[1].strip()
    
    new_data["title"].append(article["title"])
    new_data["description"].append(article["description"])
    new_data["date"].append(date)
    new_data["location"].append(crime_city)
    new_data["crime_city"].append(crime_city)
    new_data["crime_place"].append(crime_place)

# Convert the text data into numerical format using TfidfVectorizer
new_data_vectorized = vectorizer.transform(new_data["title"] + new_data["description"])

# Make predictions on the new data
new_data_predictions = model.predict(new_data_vectorized)

# Print the predictions
for i, prediction in enumerate(new_data_predictions):
    print(f"Prediction for {date}: {prediction}")

Prediction for 2023-03-20: Karnataka
Prediction for 2023-03-20: Karnataka
